In [17]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import classification_report

In [18]:
nameX = 'VehBCost'
nameY = 'VehOdo'
nameZ = 'IsBadBuy'

In [19]:
data = pd.read_csv('data.csv')
data.fillna(data.mean(), inplace=True)

In [20]:
data.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


In [21]:
data.describe()

,RefId,IsBadBuy,VehYear,VehicleAge,WheelTypeID,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VNZIP1,VehBCost,IsOnlineSale,WarrantyCost
count,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000
mean,36511.428497,0.122988,2005.343052,4.176644,1.494299,71499.995917,6128.909217,7373.636031,8497.034332,9850.928240,6132.081287,7390.681827,8775.723331,10145.385314,26345.842155,58043.059945,6730.934326,0.025280,1276.580985
std,21077.241302,0.328425,1.731252,1.712210,0.509847,14578.913128,2461.689142,2722.156232,3155.896033,3385.371986,2429.308083,2680.445479,3084.025786,3303.102877,25717.351219,26151.640415,1767.846435,0.156975,598.846788
min,1.000000,0.000000,2001.000000,0.000000,0.000000,4825.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,835.000000,2764.000000,1.000000,0.000000,462.000000
25%,18257.500000,0.000000,2004.000000,3.000000,1.000000,61837.000000,4273.000000,5407.000000,6281.000000,7494.000000,4285.000000,5425.000000,6550.000000,7796.000000,17212.000000,32124.000000,5435.000000,0.000000,837.000000
50%,36514.000000,0.000000,2005.000000,4.000000,1.494299,73361.000000,6098.000000,7305.000000,8446.000000,9790.000000,6076.000000,7328.000000,8753.000000,10114.000000,19662.000000,73108.000000,6700.000000,0.000000,1155.000000
75%,54764.500000,0.000000,2007.000000,5.000000,2.000000,82436.000000,7765.000000,9021.000000,10650.000000,12088.000000,7732.000000,9006.000000,10897.000000,12308.000000,22808.000000,80022.000000,7900.000000,0.000000,1623.000000
max,73014.000000,1.000000,2010.000000,9.000000,3.000000,115717.000000,35722.000000,36859.000000,39080.000000,41482.000000,35722.000000,36859.000000,39080.000000,41062.000000,99761.000000,99224.000000,45469.000000,1.000000,7498.000000


Оставила только возраст машины, ее марку, сколько она проехала, стоимость машины и гарантии. PRIMEUNIT и AUCGUART решила не рассматривать, так как nan составляют 98% от датасета, поэтому даже если все nan заменим на новое значение (как было на семинарах) сильного влияния не будет.

In [22]:
names = ['VehicleAge', 'Make', 'VehOdo', 'VehBCost', 'WarrantyCost']

In [23]:
X = data[names]
Y = data['IsBadBuy']

In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   VehicleAge    72983 non-null  int64  
 1   Make          72983 non-null  object 
 2   VehOdo        72983 non-null  int64  
 3   VehBCost      72983 non-null  float64
 4   WarrantyCost  72983 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 2.8+ MB


In [25]:
X = pd.get_dummies(X, columns=["Make"])

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   VehicleAge         72983 non-null  int64  
 1   VehOdo             72983 non-null  int64  
 2   VehBCost           72983 non-null  float64
 3   WarrantyCost       72983 non-null  int64  
 4   Make_ACURA         72983 non-null  uint8  
 5   Make_BUICK         72983 non-null  uint8  
 6   Make_CADILLAC      72983 non-null  uint8  
 7   Make_CHEVROLET     72983 non-null  uint8  
 8   Make_CHRYSLER      72983 non-null  uint8  
 9   Make_DODGE         72983 non-null  uint8  
 10  Make_FORD          72983 non-null  uint8  
 11  Make_GMC           72983 non-null  uint8  
 12  Make_HONDA         72983 non-null  uint8  
 13  Make_HUMMER        72983 non-null  uint8  
 14  Make_HYUNDAI       72983 non-null  uint8  
 15  Make_INFINITI      72983 non-null  uint8  
 16  Make_ISUZU         729

In [27]:
names = ['VehicleAge', 'VehOdo', 'VehBCost', 'WarrantyCost']

In [28]:
for i in names:
    max_val = max(X[i])
    X[i] = X[i] / max_val

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [30]:
alg = svm.SVC()
alg.fit(X_train, y_train)

SVC()

In [31]:
pred = alg.predict(X_test)

In [32]:
accuracy_score(pred, y_test)

0.8779630052523407

In [33]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94     19223
           1       0.00      0.00      0.00      2672

    accuracy                           0.88     21895
   macro avg       0.44      0.50      0.47     21895
weighted avg       0.77      0.88      0.82     21895



/home/crytox/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


По-разному пыталась играть с параметрами, лучше всего единички предсказывались только в этом случае.

In [34]:
alg = svm.SVC(class_weight='balanced')
alg.fit(X_train, y_train)

SVC(class_weight='balanced')

In [35]:
pred = alg.predict(X_test)

In [36]:
accuracy_score(pred, y_test)

0.6362183146837177

In [37]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.92      0.64      0.76     19223
           1       0.19      0.59      0.28      2672

    accuracy                           0.64     21895
   macro avg       0.55      0.62      0.52     21895
weighted avg       0.83      0.64      0.70     21895

